In [8]:
from google.colab import files
import pandas as pd
import io

print("Please upload your CSV file:")
uploaded = files.upload()

# Get the filename from the uploaded dictionary
for fn in uploaded.keys():
  file_name = fn
  print(f'User uploaded file "{file_name}"')

  # Read the uploaded file into a pandas DataFrame
  df = pd.read_csv(io.StringIO(uploaded[file_name].decode('utf-8')))

# Display the first 5 rows to verify it loaded correctly
display(df.head())

Please upload your CSV file:


Saving shipping_data.csv to shipping_data.csv
User uploaded file "shipping_data.csv"


,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [16]:
# 2. CLEANING & STANDARDIZATION
# Convert "1" to "Late" for readability
df['Delivery_Status'] = df['Reached_on_Time_Flag'].apply(lambda x: 'Late' if x == 1 else 'On Time')

# Rename columns for professional presentation
df.columns = ['ID', 'Warehouse_Block', 'Shipment_Mode', 'Customer_Care_Calls',
              'Customer_Rating', 'Cost_of_Product', 'Prior_Purchases',
              'Product_Importance', 'Gender', 'Discount_Offered',
              'Weight_in_gms', 'Reached_on_Time_Flag', 'Delivery_Status']

In [19]:
# 3. FEATURE ENGINEERING (The "IE" Value Add)
# Calculate Revenue at Risk (Cost of products that are late)
df['Revenue_Impact'] = df.apply(lambda x: x['Cost_of_Product'] if x['Delivery_Status'] == 'Late' else 0, axis=1)

# 4. GENERATE INSIGHTS (Save these numbers for your README!)
print("--- SUPPLY CHAIN OPTIMIZATION REPORT ---")

# Insight A: Financial Impact
total_revenue_at_risk = df['Revenue_Impact'].sum()
total_late = len(df[df['Delivery_Status'] == 'Late'])
print(f"1. FINANCIAL IMPACT: ${total_revenue_at_risk:,.2f} of inventory is currently delayed.")
print(f"   (Total Delayed Orders: {total_late})")

# Insight B: Pareto Analysis (80/20 Rule)
# Which Warehouse Block is driving the delays?
print("\n2. PARETO ANALYSIS (Late Orders by Warehouse):")
print(df[df['Delivery_Status'] == 'Late'].groupby('Warehouse_Block')['ID'].count().sort_values(ascending=False))

# Insight C: The Root Cause (Discount Strategy)
# Hypothesis: Are high-discount items being deprioritized?
high_discount_delay = df[df['Discount_Offered'] > 10]['Delivery_Status'].value_counts(normalize=True)
low_discount_delay = df[df['Discount_Offered'] < 10]['Delivery_Status'].value_counts(normalize=True)

print(f"\n3. ROOT CAUSE Analysis:")
print(f"   Late Rate for High Discount (>10%): {high_discount_delay.get('Late', 0)*100:.1f}%")
print(f"   Late Rate for Low Discount (<10%): {low_discount_delay.get('Late', 0)*100:.1f}%")

# 5. EXPORT
df.to_csv('cleaned_shipping_data.csv', index=False)
print("\n--- Success! Data exported for Tableau/SQL. ---")

--- SUPPLY CHAIN OPTIMIZATION REPORT ---
1. FINANCIAL IMPACT: $1,360,439.00 of inventory is currently delayed.
   (Total Delayed Orders: 6563)

2. PARETO ANALYSIS (Late Orders by Warehouse):
Warehouse_Block
F    2194
B    1104
D    1096
C    1094
A    1075
Name: ID, dtype: int64

3. ROOT CAUSE Analysis:
   Late Rate for High Discount (>10%): 100.0%
   Late Rate for Low Discount (<10%): 46.9%

--- Success! Data exported for Tableau/SQL. ---
